# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo= MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db=mongo.uk_food


In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments', 'users']


In [5]:
# assign the collection to a variable
establishments = db.establishments

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName','scores.Hygiene' }
results=db.establishments.find(query, fields)
#print('Following are the establishments with Hygiene score of 20')
#for i in results :
# pprint(i)



# Use count_documents to display the number of documents in the result
print("Number of documents in result:", db.establishments.count_documents(query))
#print("Number of businesses with Hygiene score 4 :", db.establishments.count_documents({'scores.Hygiene' :20}))


# Display the first document in the results using pprint
pprint(db.establishments.find_one(query))
results

Number of documents in result: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63fe9c591ca44e6abe77b0b8'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'se

In [7]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
#LondonResRatingGET4_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame result_df: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)


Rows in DataFrame result_df:  41


,_id,BusinessName,scores
0,63fe9c591ca44e6abe77b0b8,The Chase Rest Home,{'Hygiene': 20}
1,63fe9c591ca44e6abe77b43f,Brenalwood,{'Hygiene': 20}
2,63fe9c5a1ca44e6abe77b749,Melrose Hotel,{'Hygiene': 20}
3,63fe9c5a1ca44e6abe77b934,Seaford Pizza,{'Hygiene': 20}
4,63fe9c5a1ca44e6abe77b94f,Golden Palace,{'Hygiene': 20}
5,63fe9c5a1ca44e6abe77c2f1,Ashby's Butchers,{'Hygiene': 20}
6,63fe9c5a1ca44e6abe77c506,South Sea Express Cuisine,{'Hygiene': 20}
7,63fe9c5a1ca44e6abe77da30,Golden Palace,{'Hygiene': 20}
8,63fe9c5a1ca44e6abe77de91,The Tulip Tree,{'Hygiene': 20}
9,63fe9c5a1ca44e6abe77e685,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': "London"} ,'RatingValue': {'$gte': '4'}}
fields = {'BusinessName'}

results1=db.establishments.find(query, fields)
# Use count_documents to display the number of documents in the result

print("Number of documents in result:", db.establishments.count_documents(query))
#result_count= db.establishments.find(query,fields).count()
#print(result_count) 

# Display the first document in the results using pprint
pprint( db.establishments.find_one(query,fields))





Number of documents in result: 34
{'BusinessName': "Charlie's", '_id': ObjectId('63fe9c5a1ca44e6abe77caed')}


In [9]:
# Convert the result to a Pandas DataFrame
LondonResRatingGET4_df = pd.DataFrame(results1)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(LondonResRatingGET4_df))

# Display the first 10 rows of the DataFrame
LondonResRatingGET4_df.head(10)

Rows in DataFrame:  34


,_id,BusinessName
0,63fe9c5a1ca44e6abe77caed,Charlie's
1,63fe9c5a1ca44e6abe77cdfd,Mv City Cruises Erasmus
2,63fe9c5a1ca44e6abe77d94b,Benfleet Motor Yacht Club
3,63fe9c5a1ca44e6abe77e74a,Coombs Catering t/a The Lock and Key
4,63fe9c5a1ca44e6abe77e74b,Tilbury Seafarers Centre
5,63fe9c5b1ca44e6abe77ee52,Mv Valulla
6,63fe9c5b1ca44e6abe780f5a,Froottree
7,63fe9c5b1ca44e6abe780f61,Tereza Joanne
8,63fe9c5b1ca44e6abe781329,Brick Lane Brews
9,63fe9c5b1ca44e6abe781341,The Nuance Group (UK) Limited


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query5 = {'RatingValue': '5', 
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)} 
         }
sort =  [('scores.Hygiene', 1)]

# Print the results
for results in establishments.find(query5).sort(sort).limit(2):
    pprint(results)






In [11]:
# Convert result to Pandas DataFrame
top_rated_near_penang_df = pd.DataFrame(results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
#match_query = {'$match': {'scores.Hygiene': {'$et': 0}}}
# 2. Groups the matches by Local Authority
#group_query = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}
# 3. Sorts the matches from highest to lowest
#sort_query = {'$sort': { 'count': -1 }}

query = {'$match':{'scores.Hygiene' : 0}}
group = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}
sort_query = {'$sort': { 'count': -1 }}
#result = db.establishments.find(query).count()
#print(results);
pipeline = [query, group, sort_query]
# Print the number of documents in the result
results_of_Hygiene_0 = list(db.establishments.aggregate(pipeline))






# Print the number of documents in the result
print(f"There are {len(results_of_Hygiene_0)} documents in the result.")

# Print the first 10 results
pprint(results_of_Hygiene_0[0:10])

There are 55 documents in the result.
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [14]:
# Convert the result to a Pandas DataFrame
results_of_Hygiene_0_df = pd.DataFrame(results_of_Hygiene_0)


# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(results_of_Hygiene_0_df))

# Display the first 10 rows of the DataFrame
results_of_Hygiene_0_df.head(10)


Rows in DataFrame:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
